### General advice (delete this cell before submitting for review)

> * When choosing a location for your analysis, **select an area that has data on both the NCI and DEA Sandbox** to allow your code to be run on both environments. 
For example, you can check this for Landsat using the [DEA Explorer](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3/1990) (use the drop-down menu to view all products).
As of September 2019, the `DEA Sandbox` has a single year of continental Landsat data for 2015-16, and the full 1987-onward time-series for three locations (Perth WA, Brisbane QLD, and western NSW).
> * When adding **Products used**, embed the hyperlink to that specific product on the DEA Explorer using the `[product_name](product url)` syntax.
> * When writing in Markdown cells, start each sentence on a **new line**.
This makes it easy to see changes through git commits.
> * Use Australian English in markdown cells and code comments.
> * Check the [known issues](https://github.com/GeoscienceAustralia/dea-docs/wiki/Known-issues) for formatting regarding the conversion of notebooks to DEA docs using Sphinx.
Things to be aware of:
    * Sphinx is highly sensitive to bulleted lists:
        * Ensure that there is an empty line between any preceding text and the list
        * Only use the `*` bullet (`-` is not recognised)
        * Sublists must be indented by 4 spaces
    * Two kinds of formatting cannot be used simultaneously:
        * Hyperlinked code: \[\`code_format\`](hyperlink) fails
        * Bolded code: \*\*\`code_format\`\*\* fails
    * Headers must appear in heirachical order (`#`, `##`, `###`, `####`) and there can only be one title (`#`).
> * Use the [PEP8 standard](https://www.python.org/dev/peps/pep-0008/) for code. To make sure all code in the notebook is consistent, you can use the `jupyterlab_code_formatter` tool: select each code cell, then click `Edit` and then one of the `Apply X Formatter` options (`YAPF` or `Black` are recommended). This will reformat the code in the cell to a consistent style.
> * For additional guidance, refer to the style conventions and layouts in approved `develop` branch notebooks. 
Examples include
    * [Frequently_used_code/Using_load_ard.ipynb](./Frequently_used_code/Using_load_ard.ipynb)
    * [Real_world_examples/Coastal_erosion.ipynb](./Real_world_examples/Coastal_erosion.ipynb)
    * [Scripts/dea_datahandling.py](./Scripts/dea_datahandling.py)
> * The DEA Image placed in the title cell will display as long as the notebook is contained in one of the standard directories.
It does not work in the highest level directory (hence why it doesn't display in the original template notebook).
> * In the final notebook cell, include a set of relevant tags which are used to build the DEA User Guide's [Tag Index](https://docs.dea.ga.gov.au/genindex.html). 
Use all lower-case (unless the tag is an acronym), separate words with spaces (unless it is the name of an imported module), and [re-use existing tags](https://github.com/GeoscienceAustralia/dea-notebooks/wiki/List-of-tags).
Ensure the tags cell below is in `Raw` format, rather than `Markdown` or `Code`.


# Matching satellite imagery for field data <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Compatability:** Notebook currently compatible with the `NCI`|`DEA Sandbox` environment only
* **Products used:** 
[s2a_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule), 
[s2b_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2b_ard_granule)

## Background
An *optional* overview of the scientific, economic or environmental management issue or challenge being addressed by Digital Earth Australia. 
For `Beginners_Guide` or `Frequently_Used_Code` notebooks, this may include information about why the particular technique or approach is useful or required. 
If you need to cite a scientific paper or link to a website, use a persistent DOI link if possible and link in-text (e.g. [Dhu et al. 2017](https://doi.org/10.1080/20964471.2017.1402490)).

## Description
A _compulsory_ description of the notebook, including a brief overview of how Digital Earth Australia helps to address the problem set out above.
It can be good to include a run-down of the tools/methods that will be demonstrated in the notebook:

1. First we do this
2. Then we do this
3. Finally we do this

***


## Getting started
Provide any particular instructions that the user might need, e.g. To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Use standard import commands; some are shown below. 
Begin with any `iPython` magic commands, followed by standard Python packages, then any additional functionality you need from the `Scripts` directory.

In [ ]:
%matplotlib inline

import datacube
from datacube.utils.geometry import CRS, point
import dask
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import xarray as xr

sys.path.append('../Scripts')
from dea_plotting import rgb
from dea_datahandling import load_ard


### Connect to the datacube
Give your datacube app a unique name. 
Ideally, this will be the same as the notebook file name.

In [ ]:
dc = datacube.Datacube(app='DEA_notebooks_template')

### Analysis parameters

* `sample_loc_file`: Name and location of the csv file containing sampling locations with corresponding latitudes and longitudes (e.g. `../Supplementary_data/Site_matching/locations.csv`).
* `sample_data_file`: Name and location of the csv file containing sampling dates for each location and sample measurements (e.g. `../Supplementary_data/Site_matching/dates.csv`).


In [ ]:
sample_loc_file = '../Supplementary_data/Site_matching/chlorophyll_sampling_coorong_locations.csv'
sample_date_file = '../Supplementary_data/Site_matching/chlorophyll_sampling_coorong_dates.csv'

## Load the input data


In [ ]:
# Load the location csv
locations = pd.read_csv(sample_loc_file)

# View the first 5 entries
locations.head()

In [ ]:
# Load observations csv
observations = pd.read_csv('../Supplementary_data/Site_matching/chlorophyll_sampling_coorong_dates.csv')

# View the first 5 entries
observations.head()

## Identify the closest satellite data in time and space
Use markdown text for detailed, descriptive text explaining what the code below does and why it is needed.

* for each site:
    * loop over each date and extract key satellite band values and the computed NDCI at the pixel closest to the specified lat,lon of the site
    * return no observations if the closest date is more than +/-7 days out
    * write this data to an output file for each site

In [ ]:
site_dates = observations["Date"].values

products = ['s2a_ard_granule', 's2b_ard_granule']
measurements = ['nbart_red_edge_1', 'nbart_red']

# for site in locations.itertuples(index=False):
#     print(site)
    
# Extract information from locations table
site_latitude = -36.1614 #site.Lat
site_longitude = 139.6389 #site.Long
site_name = 'Coorong central lagoon 3.2km south Salt Creek' #site.SiteName

# Convert from (long, lat) to (x, y) for finding nearest pixel to site
site_point_ll = point(site_longitude, site_latitude, crs=CRS('EPSG:4326'))
site_xy = site_point_ll.to_crs(CRS('EPSG:3577')).points[0]
site_x = site_xy[0]
site_y = site_xy[1]

# Generate area to search over by adding buffer in degrees
buffer = 0.001
search_lon = (site_longitude - buffer, site_longitude + buffer)
search_lat = (site_latitude - buffer, site_latitude + buffer)

for date_string in site_dates:
    target_date = datetime.strptime(date_string, '%d/%m/%Y')
    print(target_date)

    # Load masked data from both Sentinel satellites
    ds_s2 = load_ard(dc, products=products, measurements=measurements, x=search_lon, y=search_lat, output_crs='EPSG:3577', resolution=(-10,10), lazy_load=True);

    # Isolate the timestep that is closest to the sample date
    ds_closestdate = ds_s2.sel(time=target_date, method='nearest')
    
    # Calculate the time-difference between sample date and closest date
    time_delta = np.abs(pd.to_datetime(target_date) - ds_closestdate.time.values)

    
    if time_delta < timedelta(days=7):
        ds_closestdate['ndci'] = (ds_closestdate.nbart_red_edge_1 - ds_closestdate.nbart_red)/(ds_closestdate.nbart_red_edge_1 + ds_closestdate.nbart_red)
        ds_closest_pixel = ds_closestdate.sel(x=site_x, y=site_y, method='nearest')
        ndci_value = ds_closest_pixel.ndci.values
    else:
        ndci_value = np.nan

# print(ds_closestdate.time.values)

# print(np.abs(pd.to_datetime(target_date) - ds_closestdate.time.values))


In [ ]:
print(ds_s2)

In [ ]:
observations["Date"].values[0]

In [ ]:
target_date = datetime.strptime(observations["Date"].values[30], '%d/%m/%Y')
print(target_date)

for row in locations.itertuples(index=False):
    centre_lat = row.Lat
    centre_lon = row.Long
    site_name = row.SiteName
    
    print(centre_lat)
    print(centre_lon)
    
    buffer = 0.01
    lat = (centre_lat - buffer, centre_lat + buffer)
    lon = (centre_lon - buffer, centre_lon + buffer)
    print(lat)
    print(lon)
    
    products = ['s2a_ard_granule', 's2b_ard_granule']
    
    measurements = ['nbart_red_edge_1', 'nbart_red']
    
    query = {'x': lon, 'y': lat, 'product': products, 'measurements': measurements}
    
    returned_data = dc.find_datasets(**query)
    time_delta = [target_date - dataset.center_time.replace(tzinfo=None) for dataset in returned_data]
    arg_min = np.argmin(np.abs(time_delta))
    
    print(arg_min)
    print(returned_data[arg_min].center_time)

#print(min(np.abs(time_delta)))

In [ ]:
# from dask.distributed import Client

# client = Client('dask-datacube-dask.labs:8786')

test_centre_lat = -36.1614
test_centre_lon = 139.6389

test_lat = (-36.1714, -36.1514)
test_lon = (139.6289, 139.6489)
test_date = datetime.strptime(observations["Date"].values[30], "%d/%m/%Y")

measurements = ["nbart_red_edge_1", "nbart_red"]

dask_query = {
    "x": test_lon,
    "y": test_lat,
    "product": "s2a_ard_granule",
    "measurements": measurements,
    "dask_chunks": {"time": 1},
    "output_crs": "EPSG:3577",
    "resolution": (-10, 10)
}

test_data = dc.load(**dask_query)

In [ ]:
from datacube.utils.geometry import CRS, point

In [ ]:
pnt = point(test_lon[0], test_lat[0], CRS('epsg:4326'))

In [ ]:
pnt.to_crs(CRS('epsg:3577')).points[0]

closest_lat_crs = pnt.to_crs(CRS('epsg:3577')).points[0][1]
closest_lon_crs = pnt.to_crs(CRS('epsg:3577')).points[0][0]

In [ ]:
closest = test_data.sel(time=test_date, method='nearest')
closest['ndci'] = (closest.nbart_red_edge_1 - closest.nbart_red)/(closest.nbart_red_edge_1 + closest.nbart_red)
print(closest)

# print(pd.to_datetime(test_date))
# print(closest.time.values)
# print(np.abs(pd.to_datetime(test_date) - closest.time.values))


In [ ]:
closest_pixel_1 = closest.sel(x=closest_lon_crs, y=closest_lat_crs, method='nearest')
print(closest_pixel_1.ndci.values)
closest_pixel_2 = closest.sel(x=test_lon[0], y=test_lat[0], method='nearest')
print(closest_pixel_2)

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** October 2019

**Compatible `datacube` version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)